# Escolha de Modelos e Métricas de Avaliação

Considerando o contexto da solução proposta, pensando na adequação a metodologia CRISP-DM, temos que as técnicas de NLP aqui empregadas não são aderentes aos moldes de escolha de modelo tradicionais, nem a inserção em técnicas de classificação em regressão. Devido a isso podemos pensar, como podemos avaliar quais modelos estão sendo utilizados em nossa aplicação e como podemos avaliá-los? 

Mesmo no âmbito de NLP, iremos utilizar pesos de modelos pré-treinados, assim como arquiteturas de Deep Learning, para a geração de Embeddings. Embeddings são representações vetoriais baseando-se nesse treinamento dos modelos prévios que nos devolvem o "contexto" a qual trecho de texto se refere, sendo uma técnica essencial para a solução proposta. Com isso iremos iterar sobre diferentes tipos de modelos de Embedding para avaliar qual melhor se adequa ao nosso conjunto de teste.

## Objetivos

A partir da análise exploratória dos dados, vimos que os Requerimentos apresentam muito mais informação textual e são menos propícios a falhas na recuperação que as pautas dos eventos. Devido a isso estes serão nosso foco de análise. O plano agora é verificar como podemos verificar a similaridade de um termo de pesquisa para os documentos, e para isso que iremos utilizar os Embeddings.

Em suma aqui discutimos a etapa do projeto posterior ao pré-processamento dos dados.

Após o pré-processamento como limpeza e tratamentos atrelados a NLP, devemos criar os embeddings que serão salvos em nosso banco vetorial. Para isso iremos utilizar os modelos de linguagem já prontos (treinados) e aqui avaliados. Este processo está presente no arquivo `src/data_preprocess/feature_engineering.py`.

Iremos capturar os vetores semânticos após passagem pela arquitetura de rede neural de cada modelo, resgatando os valores da última camada oculta (que apresenta maior informação semântica) a partir do primeiro Token [CLS] que contém um resumo da informação contextual de cada texto.

Esses embeddings capturados serão salvos em um banco vetorial ChromaDB a serem resgatados posteriormente.

## Conjunto de Teste

Tendo em vista que o problema deste projeto lida com dados não estruturados, para construir um "split" de teste para avaliação, foi necessário avaliar manualmente os Requerimentos legislativos e associá-los ao que seria um termo de pesquisa. Para isso foi criado dentro da pasta `data/datasets/evaluation` arquivos fixos que representem nossa ground truth, a serem aplicados nos moldes de avaliação.

Dos 60 requerimentos analisados na confecção deste projeto (retirados para o intervalo de uma semana), foram separados 21 destes para serem adequados aos dados de testes, contendo 4 temas colocados como entrada:
- "violência contra a mulher e feminicídio"
- "temas inerentes ao meio ambiente e crise climática"
- "políticas públicas de saúde",
- "estatuto da criança e do adolescente"


## Ciclo de testes de modelos

Inicialmente o projeto foi pensado com a utilização do modelo "bert-base-portuguese-cased", este é chamado de modelo BERTimbau, sendo baseado no modelo BERT muito utilizado para tarefas de NLP, porém treinado em dados da língua portuguesa. As etapas do código que apresentam a criação de Embeddings estão dispostas nos arquivos `src/data_preprocess/feature_engineering.py` que cria os embeddings a serem salvos em um banco vetorial. E em segundo temos na etapa de predição, para adequar a query enviada e buscar sua correspondência no banco, presente no arquivo `src/data_preprocess/similarity_retriever.py`. Porém se torna necessário antes de definirmos como o melhor modelo para produção, verificarmos outras opções baseando-se em métricas.

Para isso iremos iterar sobre as opções:

- "e5-base": "intfloat/multilingual-e5-base", https://huggingface.co/intfloat/multilingual-e5-base
- "bge-m3": "BAAI/bge-m3", https://huggingface.co/BAAI/bge-m3 
- "miniLM": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
- "portuguese-bert": "neuralmind/bert-base-portuguese-cased", https://huggingface.co/neuralmind/bert-base-portuguese-cased

Todas as opções vão ser acessadas e estão disponíveis a partir da biblioteca Transformers disponibilizada pelo Hugging Face.

In [1]:
%cd ..

c:\Users\renne\Documents\repos\legislativo-nlp


O funcionamento do laço de teste, além das chamadas para as funções de criação do banco vetorial e recuperação estão acopladas no arquivo `src/evaluation` para uma construção modular e voltada para a produtização deste projeto.

In [3]:
from src.evaluation import evaluate_multiple_models

A função `evaluate_multiple_models` irá ler os arquivos construídos como Golden Dataset contendo nossa ground truth de correspondência e irá iterar sobre os modelos para gerar as métricas, criando diversos bancos vetoriais e chamados as queries contendo os temas propostos.

Esta etapa é computacionalmente custosa devido a criação de múltiplos tipos de Embeddings, alguns possuem dimensões de saída um pouco maiores que outros, além de estruturas de camadas para as redes neurais diferentes.

In [4]:
evaluate_multiple_models()

Map: 100%|██████████| 59/59 [00:57<00:00,  1.02 examples/s]


INFO: Coleção limpa para ChromaDB 'e5-base'.
Ementas: 58 documentos
PDFs: 58 documentos

Avaliando a Query: 'violência contra a mulher e feminicídio'


Map: 100%|██████████| 1/1 [00:00<00:00, 17.76 examples/s]



Avaliando a Query: 'temas inerentes ao meio ambiente e crise climática'


Map: 100%|██████████| 1/1 [00:00<00:00, 15.36 examples/s]


...


Avaliando a Query: 'políticas públicas de saúde'


Map: 100%|██████████| 1/1 [00:00<00:00, 23.33 examples/s]



Avaliando a Query: 'estatuto da criança e do adolescente'


Map: 100%|██████████| 1/1 [00:00<00:00, 23.32 examples/s]




Resultados de avaliação para portuguese-bert
             ementa_baseline  pdf_baseline  ementa_llm  pdf_llm
Precision@5            0.350         0.650       0.200    0.550
Recall@5               0.345         0.668       0.134    0.601
MRR                    0.708         1.000       0.500    1.000



## Métricas utilizadas

Antes de prosseguirmos é interessante explicar brevemente quais as métricas utilizadas nessa avaliação além disso explicarmos ao que corresponde as colunas Baseline VS LLM.

### 1. Precision@5: 

"Dos 5 primeiros resultados que meu sistema retornou, qual a porcentagem de acertos?"

Mede a qualidade do topo da lista. Penaliza resultados irrelevantes mostrados ao usuário. É a métrica mais importante para medir a satisfação imediata.

Uma Precision@5 de 0.75 significa que, em média, 75% dos 5 primeiros resultados mostrados eram relevantes.
Uma Precision@5 de 0.20 significa que o usuário teve que olhar 5 resultados para encontrar apenas 1 que era relevante.

### 2. Recall@5:

"De todos os documentos que eu deveria ter encontrado, qual a porcentagem que meu sistema encontrou nos 5 primeiros resultados?"

O que ela mede: A capacidade do sistema de encontrar todos os itens relevantes. Penaliza ao deixar de fora documentos importantes. É útil quando o número total de documentos relevantes é pequeno.

Uma Recall@5 de 0.735 indica que o sistema foi capaz de encontrar, em média, 73.5% dos documentos que eram considerados corretos para as buscas, isso dentro das 5 primeiras posições.
Uma Recall@5 de 0.129 mostra que o sistema está falhando em encontrar a maioria dos documentos relevantes.

### 3. MRR (Mean Reciprocal Rank):

"Com que rapidez o usuário encontra o primeiro resultado correto?"

Mede a posição (rank) do primeiro acerto. Recompensa quando colocamos um resultado relevante logo na primeira posição.

Um MRR de 1.0 é uma pontuação perfeita. Significa que, em todas as buscas realizadas, o primeiro resultado mostrado foi um dos documentos corretos.
Um MRR de 0.5 significaria que, em média, a primeira resposta correta apareceu na segunda posição.
Um MRR de 0.3 indica que, em média, o usuário teve que descer até a terceira ou quarta posição para encontrar o primeiro item útil.



## LLM-as-a-judge

Para o propósito deste trabalho, não gostaria de realizar apenas LLMs que fizessem a avaliação da query de entrada. Porém para adequar as tecnologias mais recentes ao processo de Engenharia de IA, foi utilizados LLMs como juízes das recuperações realizadas. Ou seja esta etapa é realizada após a recuperação inicial, tanto no contexto de avaliação quanto de uso pela API.

- As recuperçaões iniciais foram feitas apenas utilizando similaridade de Cosseno entre os embeddings da query realizada e dos documentos tratados e salvos.
- Após a recuperação inicial, os textos eram passados para um LLM julgador, verificando se a posição dos itens recuperados era concisa, rerankeando os itens.

Com isso evitamos a interferência de um modelo mais custoso na recuperação, esta solução diferencia de RAGs em muito processos, como por exemplo a ausência de chunks e a não geração de respostas aumentadas, mas sim retornos diretos dos documentos e informações mais tabulares (mesmo que de origem não estruturada).

Foi utilizado a API da OpenAI para a utilização do GPT 3.5-Turbo. 

Esta lógica pode ser visualizada no arquivo `src/data_retrieval/llm_judge.py`, Este processo foi um dos mais iterativos do trabalho devido ao processo de prompt engineer necessário para criar uma query que avaliasse os resultados de forma objetiva e correta. Porém ainda é um ponto de melhoria para um melhor filtro e pente fino.

A prompt inicial foi:

```
    Você é um analista legislativo sênior, especialista em interpretar
    a relevância de documentos da Câmara dos Deputados. Sua tarefa é
    avaliar o quão bem um documento adere a um termo de pesquisa do usuário,
    fornecendo uma nota de 0 a 10.

    Tente também avaliar se o que é presente no documento está dentro do mesmo
    macro tema, ou tem intereseccionalidades com o termo pesquisado.

    Critérios de Pontuação:
    - 10: Relevância Direta: O documento tem aderência perfeita ao termo pesquisado.
    - 7-9: Relevância Alta: O documento trata do tópico principal
      que foi pesquisado, ou temáticas inerentes a ele.
    - 5-6: Relevância Parcial: O documento menciona o tópico,
      mas de forma secundária, e possui foco em outro contexto.
    - 2-4: Relevância Baixa: O documento contém poucas ou nenhuma palavras-chave
      relativo ao termo pesquisado.
    - 0-1: Irrelevante.
    """

    user_prompt = f"""
    Termos de pesquisa do Usuário:
    "{query_text}"

    Texto do Documento:
    "{document_text}"

    Sua Avaliação (responda APENAS em formato JSON com as chaves
      "score" e "justificativa"):
    """
```

A partir delas temos a geração de um Score e uma Justificativa para aquele score. Foi definido um score de corte de 7 pontos, sendo um score de certa forma rígido com a avaliação dos documentos.

Devido aos custos envolvidos, além da necessidade de criação de chaves de API, não foram testados outros LLMs nessa etapa, além de outros modelos mais recentes, porém pode ser um ponto de melhoria, possibilitando uma melhor avaliação dos documentos recuperados.

O fato do LLM agir apenas no conjunto já primeiramente filtrado reduz a quantidade de tokens enviada para a API.

## Resultados

Como resultado da avaliação temos:

| Métrica | Estratégia | `bge-m3` | `portuguese-bert` | `miniLM` | `e5-base` |
| :--- | :--- | :---: | :---: | :---: | :---: |
| **Precision@5**| Ementa (Baseline) | **0.600** | 0.350 | 0.450 | 0.550 |
| | PDF (Baseline) | **0.750** | 0.650 | 0.450 | 0.200 |
| | Ementa (LLM) | 0.300 | 0.200 | 0.300 | 0.300 |
| | PDF (LLM) | **0.700** | 0.550 | 0.450 | 0.150 |
| **Recall@5** | Ementa (Baseline) | **0.538** | 0.345 | 0.403 | 0.503 |
| | PDF (Baseline) | **0.735** | 0.668 | 0.515 | 0.129 |
| | Ementa (LLM) | **0.211** | 0.134 | **0.211** | 0.192 |
| | PDF (LLM) | **0.699** | 0.601 | 0.515 | 0.094 |
| **MRR** | Ementa (Baseline) | **0.833** | 0.708 | 0.646 | 1.000 |
| | PDF (Baseline) | **1.000** | **1.000** | 0.875 | 0.300 |
| | Ementa (LLM) | **0.750** | 0.500 | **0.750** | 0.500 |
| | PDF (LLM) | **1.000** | **1.000** | **1.000** | 0.250 |

O melhor modelo de embeddings utilizado foi o bge-m3, ele demonstrou um desempenho superior até mesmo comparado ao inicialmente idealizado (BERTimbau).

Temos também que o texto completo do requerimento em PDF se sobressai sobre o uso apenas da ementa isso sugere que, para este tipo de documento, a ementa pode ser muito concisa direta, sem muitos detalhes. O texto completo, apesar de ter mais "ruidos" e jargões técnicos, fornece um contexto semântico muito melhor para o modelo de embedding, resultando em uma busca mais precisa.


Um resultado surpreendente foi que o LLM-as-a-Judge piorou os Resultados em quase todos os cenários e para todos os modelos, as métricas após a intervenção do LLM Juiz (ementa_llm e pdf_llm) foram piores do que as da busca vetorial pura (ementa_baseline e pdf_baseline). Por exemplo para o bge-m3, a Precision@5 caiu de 0.75 para 0.70 após o julgamento. Não é uma queda tão brusca e é passível de identificar qual documento que acarretou essa mudança, talvez possuindo o tema como algo secundário e intermediário, ou com outras temáticas mais importantes sendo discutidas. O Recall@5 também caiu de 0.735 para 0.699.

Temos queo s critérios de pontuação podem ser muito rígidos ou este está sofrendo com contexto insuficiente, além do que foi comentado na seção anterior, poderiam ser utilizados modelos mais recentes e melhores para essa avaliação. Podemos também revisar o prompt para uma melhor adequação a problemática.
